## Install all required packages.

In [ ]:
### Install required packages
!pip install llama-index-embeddings-azure-openai
!pip install llama-index-llms-azure-openai
!pip install pyodbc
!pip install matplotlib
!pip install llama-index-callbacks-arize-phoenix
!pip install --upgrade --quiet llama-index-embeddings-nvidia
!pip install llama-index-core
!pip install llama-index-readers-file
!pip install llama-index-llms-nvidia
!pip install llama-index-postprocessor-nvidia-rerank
!pip install gradio

## Add observability and logging.

In [ ]:
import llama_index.core
import os

PHOENIX_API_KEY = "<YOUR_API_KEY>"
os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"api_key={PHOENIX_API_KEY}"
llama_index.core.set_global_handler(
    "arize_phoenix", endpoint="https://llamatrace.com/v1/traces"
)

In [ ]:
import logging
import sys

logging.basicConfig(
    stream=sys.stdout, level=logging.INFO
)  # logging.DEBUG for more verbose output
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Create NVIDIA endpoints

In [ ]:
## Create NVIDIA endpoints NIMS
import os
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

os.environ["NVIDIA_API_KEY"] = "<Your_API_KEY>"
llm = NVIDIA(model="meta/llama-3.1-70b-instruct")
Settings.llm = llm

In [ ]:
os.environ["py-connectionString"] = "<Your_Connection_String>"
DATABASE_SYNC_URL = os.environ["py-connectionString"]
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine
)
engine = create_engine(DATABASE_SYNC_URL)
sql_database = SQLDatabase(engine=engine
    , schema="database"
    , view_support=True
    , include_tables=[
        "kpi"
])

In [ ]:
from sqlalchemy import text
with engine.connect() as connection:
    results = connection.execute(text("SELECT TOP 100 * FROM database.kpi")).fetchall()
    print(results)

In [ ]:
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=[
        "kpi"
                        ], return_raw=True
)

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [ ]:
response = query_engine.query(
    "Summarise the top sales per shop last week."
)
print(str(response))

In [ ]:
response = query_engine.query(
    "Provide the top 5 shops weekly sales for the past 3 years?"
)
print(str(response))

## Create the functions.

In [ ]:
from llama_index.core import (
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)
persist_dir = "./.data"
if os.path.exists(persist_dir):
    print("Loading index from storage")
    load_index_from_storage(
        StorageContext.from_defaults(persist_dir=persist_dir)
    )
print("Creating index")
nodes = []
dataset = {}
with open("./dataset.json", "r") as f:
    dataset = json.load(f)

for query, response in dataset.items():
    node = TextNode(text=query)
    node.metadata["response"] = response
    node.excluded_embed_metadata_keys.append("response")
    nodes.append(node)


index = VectorStoreIndex(nodes)

query_str="Provide the top 5 shops weekly sales for the past 3 years?"
retriever = index.as_retriever(
    top_k=2
)
nodes = retriever.retrieve(query_str)
filtered_nodes = list(filter(lambda node: node.score > 0.5, nodes))
# print(filtered_nodes.__len__())
few_shot_examples = []
for node in filtered_nodes:
    query = node.text
    response = node.metadata["response"]
    few_shot_examples.append(f"Query: {query}\nResponse: {response}")
    # print("Check this line")

to_return = (
    (
        f"Below are some examples of the structure of your response:\n"
        + "\n---\n".join(few_shot_examples)
    )
    if few_shot_examples
    else ""
)

nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=[
        "kpi"
                        ]
    , return_raw=True
    , context_str_prefix=to_return
)
query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)
response = query_engine.query(query_str)
print(str(response))

## Create the Test UI

In [ ]:
import gradio as gr

def generate_text(prompt):
    # generated_text = agent.chat(prompt)
    generated_text = query_engine.query(prompt)
    return generated_text

iface = gr.Interface(fn=generate_text, inputs="text", outputs="text").launch()